# 1) Preparación previa

#### Carga de librerías

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# ES MUY PROBABLE QUE SE NECESITE INSTALAR PMDARIMA, A CONTINUACIÓN SE DEJA EL PIP INSTALL:
# !pip install pmdarima
# from pmdarima import auto_arima

import warnings
warnings.filterwarnings('ignore')

#### Lectura del dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Final%20Project/coin_Bitcoin.csv', delimiter=',')

Se realizan las modificaciones del dataset pertinentes para el análisis de series de tiempo

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.index = pd.PeriodIndex(df.Date, freq='D')
df.head()

#### Dummies

In [ ]:
df["timeIndex"] = pd.Series(np.arange(len(df['Close'])), index=df.index)

df.head()

In [ ]:
df.tail()

Se agregan las columnas necesarias

In [ ]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

Se crean las dummies

In [ ]:
dummies_mes = pd.get_dummies(df['Month'], drop_first=True, prefix='Month')
df = df.join(dummies_mes)
df.sample(10)

#### Se divide el dataset en Train y Test, usando rangos personalizados

In [ ]:
# Se utiliza este método para manejar el shape que se aplicará a los demás modelos
end_date = '2021-01-27'
mask1 = (df['Date'] <= end_date)
mask2 = (df['Date'] > end_date)

In [ ]:
# Se pasan las máscaras para obtener train y test:
df_train = df.loc[mask1]
df_test = df.loc[mask2]
print("train shape",df_train.shape)
print("test shape",df_test.shape)

#### Chequeo de que la primer fecha de test sea la siguiente al final de train:

In [ ]:
df_train.tail()

In [ ]:
df_test.head()

#### Ploteo de los dos datasets obtenidos:

In [ ]:
pd.plotting.register_matplotlib_converters()
f, ax = plt.subplots(figsize=(14,5))
df_train.plot(kind='line', x='Date', y='Close', color='blue', label='Train', ax = ax)
df_test.plot(kind='line', x='Date', y='Close', color='red', label='Test', ax = ax)
ax.legend(loc= "upper left")
plt.title('Rango para Train y para Test')
plt.show()

#### Ploteo del Target en escala logarítmica:

In [ ]:
df_train['log_value'] = np.log(df_train['Close'])
df_test['log_value'] = np.log(df_test['Close'])

In [ ]:
def plot_df(df, x, y, title = "/", xlabel='Fecha', ylabel='Valor', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi),
    plt.plot(x, y, color='tab:red'),
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel),
    plt.show()

In [ ]:
plot_df(df_train, x=df_train.Date, y=df_train['log_value'],\
    title='Log de Close del train set')

#### Entrenaiento del modelo para analizar el Summary:

In [ ]:
model_log = smf.ols('log_value ~ timeIndex',\
                          data = df_train).fit()

In [ ]:
model_log.summary()

# 2) Modelos

Se utilizará una plétora de herramientas y recursos para analizar las series de tiempo y sus implicancias. En cada paso se irá visualizando los resultados y almacenando su información para, al final de la notebook, compararlos

## a) Mean

#### Se aplica el modelo de media constante a train y test:

In [ ]:
# Se calcula el promedio:
model_mean_pred = df_train['Close'].mean()

# La predicción es fija y es la misma para el set de testeo y de entrenamiento:
df_train["Mean"] = model_mean_pred
df_test["Mean"] = model_mean_pred

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind="line", y = ["Close", "Mean"]);

In [ ]:
df_test.plot(kind="line", y = ["Close", "Mean"]);

#### Se define una función para calcular el RMSE:

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

#### Se define una función para calcular el MAPE:

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Aplicación del MAPE en Mean
model_MAPE = mean_absolute_percentage_error(df_test.Close , df_test.Mean)
print("El MAPE es ", model_MAPE)

In [ ]:
# Aplicación del RMSE en Mean
print("El RMSE es ", RMSE(df_test.Mean, df_test.Close))

#### Se guardan los resultados en un DataFrame:

El mismo será reutilizado para almacenar los resultados de los distintos modelos a utilizar

In [ ]:
df_Results = pd.DataFrame(columns = ["Model", "RMSE","MAPE"])
df_Results.loc[0, "Model"] = "Mean"
df_Results.loc[0, "RMSE"] = RMSE(df_test.Mean, df_test.Close)
df_Results.loc[0, "MAPE"] = model_MAPE
df_Results.head()

## b) RandomWalk

Se crea el shift de target en train:

In [ ]:
df_train["CloseShift"] = df_train.Close.shift()
# La primera observación va a quedar en nan, por lo que se reemplaza por el valor siguente:
df_train["CloseShift"].fillna(method='bfill', inplace=True)
df_train.head()

Se crea el shift de target en test:

In [ ]:
df_test["CloseShift"] = df_test.Close.shift()
# Se puede reemplazar el primer nan con el último valor del set de entrenamiento:
df_test.iloc[0,26] = df_train.iloc[-1,0]
df_test.head()

Lag de un período:

In [ ]:
df_train.plot(kind= "scatter", y = "Close", x = "CloseShift", s = 50);

Diferencias entre Target y el lag:

In [ ]:
df_train["Closediff"] = df_train.Close - df_train.CloseShift
df_train.Closediff.plot();

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train["RandomWalk"] = df_train.CloseShift
df_train.plot(kind="line", y = ["Close", "RandomWalk"]);

In [ ]:
df_test["RandomWalk"] = pd.Series(df_train["Close"][-1], index=df_test.index)

In [ ]:
df_test.plot(kind="line", y = ["Close", "RandomWalk"]);

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error (df_test.Close , df_test.RandomWalk)

In [ ]:
df_Results.loc[1, "Model"] = "Random Walk"
df_Results.loc[1, "RMSE"] = RMSE(df_test.RandomWalk, df_test.Close)
df_Results.loc[1, "MAPE"] = model_MAPE
df_Results

In [ ]:
# Para un resumen, se utiliza el Summary:
model_linear = smf.ols('Close ~ timeIndex', data = df_train).fit()

In [ ]:
model_linear.summary()

## c) Linear Trend

#### Se crea una columna en train con el predict:

In [ ]:
df_train["LinearTrend"] = model_linear.predict(df_train.timeIndex)

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind = "line", y = ["Close","LinearTrend"]);

#### Se repete en Test:

In [ ]:
df_test["LinearTrend"] = model_linear.predict(df_test.timeIndex)

In [ ]:
df_test.plot(kind = "line", y = ["Close","LinearTrend"]);

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error (df_test.Close , df_test.LinearTrend)

In [ ]:
df_Results.loc[2, "Model"] = "LinearTrend"
df_Results.loc[2, "RMSE"] = RMSE(df_test.LinearTrend, df_test.Close)
df_Results.loc[2, "MAPE"] = model_MAPE
df_Results

## d) Transf Log

In [ ]:
df_train['model_log'] = model_log.predict(df_train[["timeIndex"]])
df_test['model_log'] = model_log.predict(df_test[["timeIndex"]])

In [ ]:
df_train['back_model_log'] = np.exp(df_train['model_log'])
df_test['back_model_log'] = np.exp(df_test['model_log'])

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['log_value', 'model_log']);

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['Close', 'back_model_log']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['log_value', 'model_log']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['Close', 'back_model_log']);

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error (df_test.Close , df_test.back_model_log)

In [ ]:
df_Results.loc[3, "Model"] = "Transf Log"
df_Results.loc[3, "RMSE"] = RMSE(df_test['back_model_log'], df_test['Close'])
df_Results.loc[3, "MAPE"] = model_MAPE
df_Results

## e) Transf Log + Est

In [ ]:
model_log_est = smf.ols('log_value ~ timeIndex + Month_2 + Month_3 + Month_4 + Month_5 + Month_6 + Month_7 + Month_8 + Month_9 + Month_11 + Month_12',\
                          data = df_train).fit()


In [ ]:
df_train['model_log_est'] = model_log_est.predict(df_train[["timeIndex",\
                                              "Month_2", "Month_3", "Month_4", "Month_5",\
                                               "Month_6","Month_7", "Month_8", "Month_9","Month_10","Month_11","Month_12"]])


df_test['model_log_est'] = model_log_est.predict(df_test[["timeIndex",\
                                               "Month_2", "Month_3", "Month_4", "Month_5",\
                                               "Month_6","Month_7", "Month_8", "Month_9","Month_10","Month_11","Month_12"]])

In [ ]:
df_train['back_model_log_est'] = np.exp(df_train['model_log_est'])
df_test['back_model_log_est'] = np.exp(df_test['model_log_est'])

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['log_value', 'model_log_est']);

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['Close', 'back_model_log_est']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['log_value', 'model_log_est']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['Close', 'back_model_log_est']);

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error (df_test.Close , df_test.back_model_log_est)

In [ ]:
df_Results.loc[4, "Model"] = "Transf Log + est"
df_Results.loc[4, "RMSE"] = RMSE(df_test['back_model_log_est'], df_test['Close'])
df_Results.loc[4, "MAPE"] = model_MAPE
df_Results

## f) Simple Smoothing

#### Fiteo del modelo:

In [ ]:
model_exp_smoothing = SimpleExpSmoothing(df_train.Close).fit(smoothing_level=0.3, optimized=False)

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind = "line", y = "Close")
model_exp_smoothing.fittedvalues.plot();

In [ ]:
# Se define cantidad de splits:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
for train_index, val_index in tscv.split(df_train):
    print("TRAIN:", train_index, "VAL:", val_index)

Creación de una función para aplicar Cross Validation

In [ ]:
def timeseriesCVscore_exp_smoot(alpha, series):
    """
        Devuelve errores en CV  
        
        slen - longitud de la sesión para modelo Holt-Winters
    """
    # Se crea un array de errores:
    errors = []
    values = series.values
    
    # Se instancia el objeto que realiza el tscv:
    tscv = TimeSeriesSplit(n_splits=5) 
    
    # Se aplica cross validation:
    for train, test in tscv.split(values):
    
        model = SimpleExpSmoothing(values[train]).fit(smoothing_level = alpha, optimized=False)     
        predictions = model.forecast(len(test))
        actual = values[test]
    
        error = mean_squared_error(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

Aplicación de la función

In [ ]:
alphas = [0.001, 0.01, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.7]
errors = []

for alpha in alphas:
    error = timeseriesCVscore_exp_smoot(alpha, df_train.Close)
    errors.append(error)

print('Alpha óptimo:', alphas[np.argmin(errors)])

In [ ]:
model_exp_smoothing = SimpleExpSmoothing(df_train.Close).fit(smoothing_level=alphas[np.argmin(errors)], optimized=False)

In [ ]:
df_test["Simple_Smoothing"] = model_exp_smoothing.forecast(len(df_test))
df_test.head()

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error(df_test.Close , df_test.Simple_Smoothing)

In [ ]:
# Calculamos el RMSE y almacenamos los resultados
df_Results.loc[5, "Model"] = "Simple Smoothing"
df_Results.loc[5, "RMSE"] = RMSE(df_test["Simple_Smoothing"], df_test.Close)
df_Results.loc[5, "MAPE"] = model_MAPE

df_Results

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_test.plot(kind="line", y = ["Close", "Simple_Smoothing"]);

## g) Dickey Fuller + Autocorrelación

#### Se prueba si los residuos son estacionarios

In [ ]:
residuo = df_train['Close'] - df_train['back_model_log_est']
plt.plot(df_train.timeIndex, residuo, '-');

#### Aplicación de Dickey Fuller al residuo: 

In [ ]:
result = adfuller(residuo)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in  result[4].items():
    print('Valor crítico %s: %.2f' % (key,value))

No se puede rechazar la H0 con un nivel de significación del 5%.

In [ ]:
# Se prueba ahora con los residuos antes de realizar back transform:
res_log_est = df_train['log_value'] - df_train['model_log_est']
plt.plot(df_train.timeIndex, res_log_est, '-');

#### Segundo testeo de la estacionalidad de los residuos:

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(res_log_est)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in  result[4].items():
    print('Valor crítico %s: %.2f' % (key,value))

Aún no se puede rechazar la H0, pero se continúa el análisis

In [ ]:
# Cálculo del res_log con 20 rezagos:
lag_acf = acf(res_log_est, nlags = 20)
lag_acf

#### Se almacena la serie ACF y se plotea:

In [ ]:
ACF = pd.Series(lag_acf)
ACF.plot(kind = "bar");

#### Se repiten los pasos anteriores pero con PACF:

In [ ]:
lag_pacf = pacf(res_log_est, nlags=20, method='ols');

In [ ]:
PACF = pd.Series(lag_pacf)
PACF.plot(kind = "bar");

Se puede concluir de este análisis que la correlación indirecta es alta, pero la parcial que considera solo influencia directa de cada período es absoluta en tan solo un mes antes del momento a analizar: esto habla de la alta volatilidad del caso a analizar, ya que se deduce de la herramienta y los datos que es de poca utilidad para la predicción de valores en las criptomonedas la utilización de información de más de un mes atrás

#### Se crea una función para plotear una serie con información sobre los ACF y PACF y su estacionalidad:

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """ 
        Plotea la serie de tiempo, el ACF y PACF y el test de Dickey–Fuller
        
        y - serie de tiempo
        lags - cuántos lags incluir para el cálculo de la ACF y PACF
        
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        
        # Se definen ejes
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        
        # Se obtiene el p-value con H0: raiz unitaria presente
        p_value = sm.tsa.stattools.adfuller(y)[1]
        
        ts_ax.set_title('Análisis de la Serie de Tiempo\n Dickey-Fuller: p={0:.5f}'\
                        .format(p_value))
        
        # Plot de autocorrelacion
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        # Plot de autocorrelacion parcial
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
# Aplicación de la función con la serie res_log:
tsplot(res_log_est, lags=36)

## h) ARIMA

#### Se aplica el auto_arima sobre res_log_est

In [ ]:
stepwise_fit = auto_arima(res_log_est, trace=True, suppress_warnings=True)

#### Como se obtuvo mejores resultados con p=2 y q=1, s aplica:

In [ ]:
model_ARIMA = ARIMA(res_log_est, order=(2,0,1))

# Estimación del modelo:
results_ARIMA = model_ARIMA.fit()
results_ARIMA.fittedvalues.head()

#### Se observa el summary:

In [ ]:
print(results_ARIMA.summary())

#### Ploteo de resultados:

In [ ]:
plt.figure(figsize=(7,3.5))
res_log_est.plot()
results_ARIMA.fittedvalues.plot();

In [ ]:
results_ARIMA.plot_predict(end=len(df['Close']));

#### Análisis de los residuos del modelo ARIMA:

In [ ]:
res_ARIMA =  results_ARIMA.fittedvalues - res_log_est

In [ ]:
tsplot(res_ARIMA, lags=36)

#### Aplicación del método Forecast:

In [ ]:
predictions_ARIMA, se, conf = results_ARIMA.forecast(len(df_test['Close']), alpha=0.05)

In [ ]:
# Se crea una variable en train y test y se plotea los resultados:

df_train['log_model_ARIMA'] = df_train['model_log_est'] + results_ARIMA.fittedvalues

df_test['log_model_ARIMA'] = df_test['model_log_est'] + predictions_ARIMA

#### Ploteo de las predicciones vs las series reales, en train y test:

In [ ]:
df_train.plot(kind = "line", y = ['log_value', 'log_model_ARIMA']);

In [ ]:
df_test.plot(kind = "line", y = ['log_value', 'log_model_ARIMA']);

In [ ]:
# Se crea una variable en train y test con back transformation del modelo log y se plotea resultados:

df_train['back_log_model_ARIMA'] = np.exp(df_train['log_model_ARIMA'])

df_test['back_log_model_ARIMA'] = np.exp(df_test['log_model_ARIMA'])

In [ ]:
df_train.plot(kind = "line", y = ['Close', 'back_log_model_ARIMA']);

In [ ]:
df_test.plot(kind = "line", y = ['Close', 'back_log_model_ARIMA']);

#### Se calcula el MAPE + RMSE y se almacena

In [ ]:
model_MAPE = mean_absolute_percentage_error (df_test.Close , df_test.back_log_model_ARIMA)

In [ ]:
df_Results.loc[6, "Model"] = "Log Model + est + ARIMA"
df_Results.loc[6, "RMSE"] = RMSE(df_test['back_log_model_ARIMA'], df_test['Close'])
df_Results.loc[6, "MAPE"] = model_MAPE

df_Results

# 3) Comparación de resultados

#### Análisis de RMSE y MAPE en cuadro

Como el Prophet se aplica en la siguiente notebook, se insertan los resultados manualmente a continuación:

In [ ]:
df_Results.loc[7, "Model"] = "Prophet"
df_Results.loc[7, "RMSE"] = "6997.16"
df_Results.loc[7, "MAPE"] = "15.54"

df_Results

#### Análisis de RMSE y MAPE visualizado

In [ ]:
plt.figure(figsize=(18, 10)).suptitle('Comparación de rendimiento', y=0.3, fontsize = 16, color='white', backgroundcolor='gray')
plt.plot(df_Results.RMSE, color='red', linewidth=2, label = "RMSE")


plt.ylabel('Rendimiento', fontsize=18)
plt.legend(fontsize = 15)

axes= plt.gca()
ymin= 0
ymax= 50000
axes.set_ylim([ymin, ymax])

plt.axvline(x='Means', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Random Walk',color="grey", linestyle="--", lw=1.3)
plt.axvline(x='LinearTrend', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Transf Log' , color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Transf Log + est', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Simple Smoothing', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Log_Model_est_ARIMA_plot', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Prophet', color="grey", linestyle="--", lw=1.3)

plt.grid(which='major', axis='y', color='black', lw=0.4, alpha=0.6)
plt.show()

In [ ]:
plt.figure(figsize=(18, 10)).suptitle('Comparación de rendimiento', y=0.3, fontsize = 16, color='white', backgroundcolor='gray')
plt.plot(df_Results.MAPE, color='b', linewidth=2, label = "MAPE")


plt.ylabel('Rendimiento', fontsize=18)
plt.legend(fontsize = 15)

axes= plt.gca()
ymin= 0
ymax= 99
axes.set_ylim([ymin, ymax])

plt.axvline(x='Means', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Random Walk',color="grey", linestyle="--", lw=1.3)
plt.axvline(x='LinearTrend', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Transf Log' , color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Transf Log + est', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Simple Smoothing', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Log_Model_est_ARIMA_plot', color="grey", linestyle="--", lw=1.3)
plt.axvline(x='Prophet', color="grey", linestyle="--", lw=1.3)

plt.grid(which='major', axis='y', color='black', lw=0.4, alpha=0.6)
plt.show()

En la 3ra y última notebook se prueba una herramienta no vista en el curso. Se separó para ser planteada como un anexo, debido a nuestra incertidumbre de sus resultados ya que no fue trabajada en clase.